In [ ]:
%pip install -i https://test.pypi.org/simple/ --extra-index-url=https://pypi.org/simple azure-datalake-utils==0.2.0.dev23

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip uninstall azure-datalake-utils -y

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip show azure-datalake-utils

Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys

import pandas as pd

sys.path.append('../')

In [2]:
from azure_datalake_utils import Datalake

In [3]:
import json

with open('../keys.json') as f:
    creds = json.load(f)


## reproducir el caso `BadZipFile`

Se observa un comportamiento extraño bajo estas condiciones:

1. Se importa la libreria en un sesion de python.
1. Se abre el archivo de excel y se reemplaza en el datalake.
1. Usando la misma sesión de python, se trata de leer el archivo con `dl.read_excel("hacebanalitica-user-servicio/configuracion_valores_salesforce.xlsx")`
1. Una exepción `BadZipFile: File is not a zip file`.

WorArounds:
1. Reiniciar la sesión de python. Solo realizar esa acción ya el error no aparece.
2. usar el `flag=True` ejemplo:

```python
excel = dl.read_excel("hacebanalitica-user-servicio/configuracion_valores_salesforce.xlsx", sheet_name = 'familia', experimental = True)
```
Este flag es experimental y es una solución rapida para entornos donde reiniciar python de manera periodica no es una opción (ejemplo an Azure Functions.)

In [5]:
dl = Datalake.from_account_key(**creds)

In [ ]:
# modificar el archivo de excel a leer y subirlo a la misma ruta.
excel = dl.read_excel("hacebanalitica-user-servicio/configuracion_valores_salesforce.xlsx", sheet_name = 'familia')

In [15]:
# si es usa el flag, ya no hay falla.
excel = dl.read_excel("hacebanalitica-user-servicio/configuracion_valores_salesforce.xlsx", sheet_name = 'familia', experimental = True)

## Uso normal.

In [6]:
df = pd.DataFrame({"foo_str": ['bar\n', 'foo,', 'bar|'], "bar_str": ["bar", "foo|", "bar\r"], "foo": [1, 2, 3]})

In [9]:
dl.write_csv(df, 'hacebanalitica-user-cientificos/prueba/test.csv', sep='|')

In [8]:
test = dl.read_csv('hacebanalitica-user-cientificos/prueba/*.csv', header = None)

In [9]:
test = dl.read_csv('hacebanalitica-curated-ventas/ventas/haceb/ventas/ventas_2022.csv', sep=';')

/home/german/repos/azure-datalake-utils/docs/../azure_datalake_utils/azure_datalake_utils.py:80: DtypeWarning: Columns (11,15,16,17,38,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"az://{ruta}", storage_options=self.storage_options, **kwargs)


In [11]:
test.head()

,NdeFactur,ClasedeFactura,DescripcionFactura,Clase de Pedido,Descripcion ClasePedido,CodigoMaterial,NombreMaterial,Cantidad,EstadodeContabilizacion,CodigoPagador,...,Oficina de Ventas,Descripción Oficina de Ventas,Margen,Sociedad,Org Ventas,Grupo de Comisiones,Descripción Grupo de Comisiones,Descripción Nivel Privilegio,Email_ZPOS,Habeas_Data
0,107T1471,ZSCP,Factura de Venta,ZSCI,Ped POS Contado EI,2,SERVICIO CALIFICADO,"0,5",C,800104,...,OF02,Centro,"10000,0",1000,1000,60,SAC,NaN,NaN,NaN
1,107T1472,ZSCP,Factura de Venta,ZSCI,Ped POS Contado EI,496,INSTALACION GRATUITA,"1,0",C,800104,...,OF02,Centro,"10000,0",1000,1000,60,SAC,NaN,,"2,0"
2,107T1472,ZSCP,Factura de Venta,ZSCI,Ped POS Contado EI,1001849,CODO CALLE 1/2' GALVANIZADO,"1,0",C,800104,...,OF02,Centro,"5129,0",1000,1000,50,Repuestos y Accesori,NaN,,"2,0"
3,107T1472,ZSCP,Factura de Venta,ZSCI,Ped POS Contado EI,1005211,MANGUERA ESPIRAL ENCAUCHADA 36,"1,0",C,800104,...,OF02,Centro,"5792,0",1000,1000,50,Repuestos y Accesori,NaN,,"2,0"
4,101T370,ZSCP,Factura de Venta,ZSCI,Ped POS Contado EI,1006119,BIMETALICO FSTB NEV 611 SE RD21,"1,0",C,800104,...,OF02,Centro,"6534,0",1000,1000,50,Repuestos y Accesori,NaN,NaN,NaN


In [12]:
test = dl.read_csv('hacebanalitica/raw/sap/ztsd_clientepos/load_date=2022-05-04/ztsd_clientepos.csv', sep='|')

/home/german/repos/azure-datalake-utils/docs/../azure_datalake_utils/azure_datalake_utils.py:80: DtypeWarning: Columns (10,16,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"az://{ruta}", storage_options=self.storage_options, **kwargs)


In [14]:
test.head()

,MANDT,FITYP,STCDT,STCD1,KUNNR,ANRED,NAME1,STREET,CITY1,COUNTRY,...,ERNAM,AEDAT,AEZET,UNAM,IN_CLISAP,CSTCNDGRP1,ZSD_HABEASDATA,TRANSPZONE,LANDX,KTOKD
0,300,PN,13,632453,632453,Señor(a),GUSTAVO PARRA,CR 45 A 80 SUR 75,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,
1,300,PN,13,63311387,63311387,Señor(a),ALMANZAR SANCHEZ,CORDOBA,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,
2,300,PN,13,6819796,6819796,Señor(a),FERNANDO BOTERO,CR 81 B 7 A 49,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,
3,300,PN,13,70030346,70030346,Señor(a),ALBERTO LONDONO,CLIENTE TRNASPORTA,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,
4,300,PN,13,70040014,70040014,Señor(a),PEDRO HERNANDEZ,COTIZACION,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,


In [13]:
## Interactivo

In [15]:
dl = Datalake('datalakeanaliticabi', '5368fc5a-2692-4a6a-b226-e2802e94e22b')

In [16]:
test = dl.read_csv('hacebanalitica-user-cientificos/prueba/*.csv', header = None)

In [ ]:
test = dl.read_csv('hacebanalitica/raw/sap/ztsd_clientepos/load_date=2022-05-04/ztsd_clientepos.csv', sep='|')

## Probar el uso del FS

In [4]:
from adlfs import AzureBlobFileSystem

In [5]:
fs = AzureBlobFileSystem(account_name=creds['datalake_name'], account_key=creds['account_key'])

In [8]:
fs.find("hacebanalitica/raw/tuya_cloud/devices/")[0:10]

['hacebanalitica/raw/tuya_cloud/devices/product_name=/start_date=2022_10_30/load_date=2022_10_31/devices.csv',
 'hacebanalitica/raw/tuya_cloud/devices/product_name=/start_date=2022_10_31/load_date=2022_11_01/devices.csv',
 'hacebanalitica/raw/tuya_cloud/devices/product_name=/start_date=2022_11_01/load_date=2022_11_02/devices.csv',
 'hacebanalitica/raw/tuya_cloud/devices/product_name=/start_date=2022_11_02/load_date=2022_11_03/devices.csv',
 'hacebanalitica/raw/tuya_cloud/devices/product_name=/start_date=2022_11_03/load_date=2022_11_04/devices.csv',
 'hacebanalitica/raw/tuya_cloud/devices/product_name=/start_date=2022_11_04/load_date=2022_11_05/devices.csv',
 'hacebanalitica/raw/tuya_cloud/devices/product_name=/start_date=2022_11_05/load_date=2022_11_06/devices.csv',
 'hacebanalitica/raw/tuya_cloud/devices/product_name=/start_date=2022_11_06/load_date=2022_11_07/devices.csv',
 'hacebanalitica/raw/tuya_cloud/devices/product_name=/start_date=2022_11_07/load_date=2022_11_08/devices.csv',
 

In [8]:
for f in fs.walk("hacebanalitica/raw/tuya_cloud/devices/product_name=A19 RGB+CCT WIFI+Bluetooth/start_date=2022_09_24/", detail= True):
    print(f)

('hacebanalitica/raw/tuya_cloud/devices/product_name=A19 RGB+CCT WIFI+Bluetooth/start_date=2022_09_24/', {'load_date=2022_10_24': {'name': 'hacebanalitica/raw/tuya_cloud/devices/product_name=A19 RGB+CCT WIFI+Bluetooth/start_date=2022_09_24/load_date=2022_10_24', 'size': None, 'type': 'directory'}}, {})
('hacebanalitica/raw/tuya_cloud/devices/product_name=A19 RGB+CCT WIFI+Bluetooth/start_date=2022_09_24/load_date=2022_10_24', {}, {'devices.csv': {'metadata': {'is_directory': 'false'}, 'creation_time': datetime.datetime(2022, 10, 25, 1, 31, 47, tzinfo=datetime.timezone.utc), 'deleted': None, 'deleted_time': None, 'last_modified': datetime.datetime(2022, 10, 25, 1, 31, 47, tzinfo=datetime.timezone.utc), 'content_settings': {'content_type': 'application/octet-stream', 'content_encoding': None, 'content_language': None, 'content_md5': None, 'content_disposition': None, 'cache_control': None}, 'remaining_retention_days': None, 'archive_status': None, 'last_accessed_on': datetime.datetime(202

In [14]:
dirs = fs.listdir("hacebanalitica/processed/ventas/sabana_pos/start_date=2022-01-01/end_date=2022-04-30/")
dirs = [d for d in dirs if d.get('type') == 'directory']

In [33]:
sorted(files.items(), key=lambda x:x[1], reverse=True)[0][0]

'hacebanalitica/processed/ventas/sabana_pos/start_date=2022-01-01/end_date=2022-04-30/load_date=2022-11-16/sabana_pos.csv'

In [29]:
sorted(files, key=lambda a: a.last_modified, reverse=False)

{'hacebanalitica/processed/ventas/sabana_pos/start_date=2022-01-01/end_date=2022-04-30/load_date=2022-11-14/sabana_pos.csv': datetime.datetime(2022, 11, 15, 5, 15, 55, tzinfo=datetime.timezone.utc),
 'hacebanalitica/processed/ventas/sabana_pos/start_date=2022-01-01/end_date=2022-04-30/load_date=2022-11-15/sabana_pos.csv': datetime.datetime(2022, 11, 16, 5, 57, 50, tzinfo=datetime.timezone.utc),
 'hacebanalitica/processed/ventas/sabana_pos/start_date=2022-01-01/end_date=2022-04-30/load_date=2022-11-16/sabana_pos.csv': datetime.datetime(2022, 11, 16, 19, 1, 29, tzinfo=datetime.timezone.utc)}

In [45]:
cli = fs.service_client

In [51]:
from azure.storage.blob import  BlobServiceClient

In [62]:
get_last_modified("hacebanalitica", "raw/tuya_cloud/devices/product_name=A19 RGB+CCT WIFI+Bluetooth/")

'raw/tuya_cloud/devices/product_name=A19 RGB+CCT WIFI+Bluetooth/start_date=2022_11_28/load_date=2022_11_29/devices.csv'

In [19]:
fs.ls("hacebanalitica/raw/tuya_cloud/devices/product_name=A19 RGB+CCT WIFI+Bluetooth/start_date=2022_09_24/")

['hacebanalitica/raw/tuya_cloud/devices/product_name=A19 RGB+CCT WIFI+Bluetooth/start_date=2022_09_24/load_date=2022_10_24']